In [44]:
import numpy as np
import json
from pathlib import Path
import os
import requests
from datetime import datetime
import datetime as dtt

def get_returns(candle):
    rendements = []
    for j in range(len(candle) - 1):
        j1 = candle[j]
        j2 = candle[j + 1]
        cash_flow = 0
        if "cash_flow" in j2:
            cash_flow = j2["cash_flow"]
        r = (j2["close"] - j1["close"] + cash_flow)/j1["close"]
        rendements.append(r)
    return np.array(rendements)
    
def get_vol(candle):
    closes = np.array([x["open"] for x in candles])
    return get_returns(candles).var() * 100
    
def get_volatility(id, start, end):
    param = {"ratio": [18], "asset": [id], "start_date": start, "end_date": end}
    s = requests.Session()
    s.auth = ('epita_user_6', 'dolphin39833')
    s.body = param
    r = s.post("https://dolphin.jump-technology.com:3389/api/v1/ratio/invoke", data = json.dumps(param), verify=False)
    r = r.json()
    print(r)
    
    
def pull_data():
    s = requests.Session()
    s.auth = ('epita_user_6', 'dolphin39833')
    r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset", verify=False)
    data = r.json()
    ids = [a["ASSET_DATABASE_ID"] for a in data]
    for i in ids:
        print(i["value"])
        p = Path("data/raw/" + i["value"])
        if not p.exists():
            f = open("data/raw/" + i["value"], "w")
            r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset/" + i["value"] + "/quote", verify=False)
            candles = r.json()
            f.write(json.dumps(candles))
            f.close()
    
    
def dateInPeriod(date, start, end):
    #start = datetime.strptime(start, "%Y-%m-%d")
    #end = datetime.strptime(end, "%Y-%m-%d")
    if date >= start and date <= end:
        return True
    return False
    
    
def cleanData(path):
    fi = open(path, "r")
    data = fi.readlines()[0]
    data = json.loads(data)

    start_date = data[0]["date"]
    end_date = data[-1]["date"]
    
    cleaned = [[data[0]["date"], data[0]["close"]]]
    prev = datetime.strptime(data[0]["date"], "%Y-%m-%d")

    for e in data[1:]:
        diff = (datetime.strptime(e["date"], "%Y-%m-%d") - prev).days
        prev_val = cleaned[-1][1]
        for i in range(1, diff + 1):
            prev = prev + dtt.timedelta(days=1)
            tmp = [datetime.strftime(prev, "%Y-%m-%d"), prev_val + i * (e["close"] - prev_val) / diff]
            cleaned.append(tmp)
        prev = datetime.strptime(e["date"], "%Y-%m-%d")
        
    fo = open("data/cleaned/" + path.split("/")[-1], "w")
    fo.write(json.dumps(cleaned))
    
def getMatrix(date_start, date_end):
    mat = []
    for root, dirs, files in os.walk("data/"):
        for f in files:
            if not f.startswith("."):
                fi = open(os.path.join(root, f))
                v = fi.readlines()[0]
                v = json.loads(v)
                ds = datetime.strptime(v[0]["date"], "%Y-%m-%d")
                de = datetime.strptime(v[-1]["date"], "%Y-%m-%d")
                if ds <= date_start and de >= date_end:
                    v = cleanData([[x["date"], float(x["close"])] for x in v], date_start, date_end)
                    v = np.array(v)
                    mat.append(v)
                #else:
                #    print("Can't take file ", f, " because starts at ", ds, " and end at ", de)
                fi.close()
               
    return np.array(mat)


#pull_data()
    
#def volatility(candle):
#def sharpe(candle):

In [47]:
import requests

s = requests.Session()
s.auth = ('epita_user_6', 'dolphin39833')
r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset", verify=False)
data = r.json()
ids = [a["ASSET_DATABASE_ID"] for a in data]
i = ids[0]
r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset/" + i["value"] + "/quote", verify=False)
candles = r.json()

/Users/Antoine/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Antoine/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'value': '315', 'type': 'int32'}


In [45]:
pull_data()
for root, dirs, files in os.walk("data/raw/"):
        for f in files:
            if not f.startswith("."):
                cleanData(os.path.join(root, f))


/Users/Antoine/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


315
405
61
67
16
144
70
234
376
109
31
68
169
248
410
494
489
345
508
471
483
481
482
371
350
147
384
333
37
341
153
307
42
161
209
286
74
77
349
346
368
386
347
490
442
530
276
5
168
285
224
51
356
202
305
110
233
273
63
54
49
52
408
528
537
28
441
466
411
422
416
418
452
173
318
100
510
363
511
225
6
207
294
171
316
545
430
544
464
57
154
549
103
201
262
130
9
232
163
15
391
364
159
86
208
29
191
390
531
467
498
532
409
526
470
20
311
26
214
203
245
440
156
48
12
117
23
426
195
216
47
17
423
236
252
59
215
337
134
157
342
497
488
533
461
123
107
84
66
403
529
541
348
425
379
527
462
53
72
162
165
182
221
298
65
308
140
293
255
265
267
331
170
30
7
82
261
493
60
180
534
509
552
413
457
515
517
357
427
85
39
141
280
314
41
535
484
383
374
353
460
373
14
179
71
1
106
13
97
164
211
235
185
88
512
392
477
87
143
240
367
249
351
428
424
500
419
514
525
429
432
435
501
524
475
129
243
178
445
378
359
188
226
270
301
113
313
133
229
380
242
199
21
45
538
507
506
557
437
148
197
281
36
152
4
